## DeepDTI-Affinity: Predicting Drug-Target Binding Affinity

* **Author:** Hosein Mohammadi
* **Date:** July 2024
* **Contact:** [huseinmohammadi83@gmail.com](mailto:huseinmohammadi83@gmail.com)
* **LinkedIn:** [Hosein Mohammadi](https://www.linkedin.com/in/hosein-mohammadi-979b8a2b2/)
* **Project Repository:** [DeepDTI-Affinity](https://github.com/Hosein541/DeepDTI-Affinity)
---

### Project Overview

This notebook implements an end-to-end deep learning pipeline to predict the binding affinity ($pK_d$) between drug molecules and protein targets. The project tackles the problem using a sophisticated **multi-modal architecture** and advanced, data-centric training strategies.

The final model successfully overcomes the challenges of a highly imbalanced dataset to achieve a **Pearson Correlation of ~0.82** on the Davis dataset, demonstrating a strong predictive capability.



### Key Features & Techniques Implemented:
* **Multi-Modal Architecture:** A two-branch network that simultaneously processes molecular graphs and protein sequences.
* **Graph Attention Network (GAT):** A powerful graph neural network used to encode drug molecules from their SMILES representation.
* **Deep 1D Residual CNN:** A deep convolutional neural network with residual connections to effectively learn features from protein amino acid sequences.
* **Advanced Data Handling:** The project identifies and solves a severe data imbalance problem in the regression task using a SMOTE-like oversampling technique.
* **Two-Stage Fine-Tuning:** A professional workflow involving:
    1.  Pre-training the encoders on the original dataset.
    2.  Generating synthetic data in the learned embedding space.
    3.  Freezing the encoders and fine-tuning the final classifier head on the balanced dataset.

In [ ]:
# Cell 1: Project Dependencies Installation
!pip install rdkit-pypi torch_geometric smogn -q

In [ ]:
# Cell 2: Main Library Imports

# Standard Libraries
import os
import requests
import tarfile
import zipfile
from urllib.parse import urlparse

# Core Data Science & ML Libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Cheminformatics
from rdkit import Chem

# PyTorch Core
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Note the alias to prevent name collision
from torch.utils.data import TensorDataset, DataLoader as TorchDataLoader 

# PyTorch Geometric
from torch_geometric.data import Data
# Note the alias to prevent name collision
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch_geometric.nn import GATConv, global_mean_pool

### Part 1: Data Preparation

This initial section handles all the necessary data preprocessing steps to prepare the Davis dataset for our regression model. The process involves two main stages:

#### 1.1 Dataset Download and Extraction
First, we programmatically download the compressed Davis dataset from its source. The script handles the download and extraction, placing the raw `.txt` file into a local directory. As the direct link can expire, it's designed for the user to paste the latest link from the source website.

#### 1.2 Graph Conversion and Data Structuring
This is the core preprocessing step for our model. We iterate through the raw dataset and perform the following actions for each drug-target pair:
* **SMILES to Graph:** The drug's SMILES string is converted into a molecular graph representation using the `RDKit` library. This graph includes atom features (nodes) and bond information (edges).
* **Data Object Creation:** Each processed pair is stored in a `torch_geometric.data.Data` object. This object conveniently holds the drug's graph structure, the corresponding protein's amino acid sequence, and the **continuous binding affinity ($pK_d$)** as the target label (`y`).
* **Final Dataset:** The complete list of these `Data` objects is saved to a single file (`davis_regression_dataset.pt`), which will be loaded directly by our model in the subsequent training phases.

In [ ]:
# --- Step 1: Download and Extract the Dataset ---

# Please paste the direct download link you copied from the website here.
dataset_url = "PASTE_THE_CORRECT_DOWNLOAD_LINK_HERE"

# Determine the filename from the URL
parsed_url = urlparse(dataset_url)
filename = os.path.basename(parsed_url.path)
extract_folder = 'davis_dataset'

print(f"Starting download of '{filename}'...")

# Check if a URL has been provided
if dataset_url == "PASTE_THE_CORRECT_DOWNLOAD_LINK_HERE":
    print("Error: Please update the 'dataset_url' variable with the correct download link.")
else:
    # Send a GET request to the URL
    response = requests.get(dataset_url, stream=True)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Save the downloaded file
        with open(filename, 'wb') as f:
            f.write(response.content)
        print("Download completed successfully.")

        # --- Step 2: Extract the Dataset ---
        print(f"Extracting file into the '{extract_folder}' directory...")

        # Create the extraction directory if it doesn't exist
        if not os.path.exists(extract_folder):
            os.makedirs(extract_folder)

        # Extract based on file type
        try:
            if filename.endswith('.tar.gz'):
                with tarfile.open(filename, 'r:gz') as tar:
                    tar.extractall(path=extract_folder)
                print("'.tar.gz' file extracted successfully.")
            # Note: Python's standard library doesn't handle .rar.
            # If the file is .zip, this will work. For .rar, you'd need an external library like 'unrar'.
            elif filename.endswith('.zip'):
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(extract_folder)
                print("'.zip' file extracted successfully.")
            else:
                print(f"Warning: File type '{filename.split('.')[-1]}' might not be extractable with this script.")
                print("Please extract it manually.")

            # List the contents of the main extracted directory for verification
            # The actual data is often inside a nested folder, e.g., 'davis_dataset/davis'
            main_extracted_dir = os.path.join(extract_folder, os.listdir(extract_folder)[0])
            if os.path.isdir(main_extracted_dir):
                print("\nFiles found in the dataset directory:")
                for item in os.listdir(main_extracted_dir):
                    print(f"- {item}")

            # Clean up the downloaded compressed file
            os.remove(filename)
            print(f"\nTemporary file '{filename}' has been deleted.")

        except Exception as e:
            print(f"An error occurred during extraction: {e}")

    else:
        print(f"Error during download. HTTP Status Code: {response.status_code}")

In [ ]:
def smiles_to_graph(smiles_string):
    """Converts a SMILES string to a PyTorch Geometric Data object."""
    try:
        mol = Chem.MolFromSmiles(smiles_string)
        if mol is None: return None
        atom_features = [[
            atom.GetAtomicNum(), atom.GetFormalCharge(), atom.GetDegree(),
            int(atom.GetIsAromatic()), int(atom.GetHybridization())
        ] for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float)

        edge_indices = []
        for bond in mol.GetBonds():
            i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            edge_indices.extend([(i, j), (j, i)])
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

        return Data(x=x, edge_index=edge_index)
    except Exception:
        return None

# --- Main Script for Regression Data Preparation ---
print("--- Preparing Data for Regression Task ---")
try:
    # Load the original dataset with affinity values
    df = pd.read_csv('/content/davis_dataset/davis.txt', sep=' ', header=None, names=["Drug Id", "Protein Id", "smiles", "sequence", "affinity"])

    regression_data_list = []

    # Iterate over each row to create graph objects
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Regression Data"):
        graph = smiles_to_graph(row['smiles'])

        # We only proceed if the SMILES string was valid
        if graph is not None:
            # Attach the protein sequence
            graph.sequence = row['sequence']

            # KEY CHANGE: The label 'y' is now the continuous affinity value
            graph.y = torch.tensor([row['affinity']], dtype=torch.float)

            regression_data_list.append(graph)

    print(f"\nProcessing complete.")
    print(f"Created {len(regression_data_list)} valid data points for regression.")

    # Save the final list of graph objects for the regression task
    torch.save(regression_data_list, 'davis_regression_dataset.pt')
    print("Regression dataset saved to 'davis_regression_dataset.pt'")

    # Verification
    print("\n--- Verifying first data point ---")
    first_point = regression_data_list[0]
    print(first_point)
    print(f"Label (Affinity Value): {first_point.y.item()}")

except FileNotFoundError:
    print("Error: 'davis.txt' not found. Please ensure the file is in the correct directory.")
except Exception as e:
    print(f"An error occurred: {e}")

### Part 2: Model Architecture

The model is composed of three main PyTorch modules: two encoders to process each input modality (drugs and proteins) and a final interaction model to produce the affinity prediction.

#### 2.1 Drug Encoder: Graph Attention Network (GAT)

To capture the complex topological information of the drug molecules, we use a **Graph Attention Network (GAT)**. Unlike standard graph convolutions, GAT layers use a self-attention mechanism, allowing the model to dynamically weigh the importance of different neighboring atoms when constructing the molecule's feature representation.

The encoder consists of a stack of `GATConv` layers that progressively transform the initial atom features into a single, fixed-size embedding vector for the entire molecule using a global pooling layer.



#### 2.2 Protein Encoder: Deep 1D Residual CNN

For the protein sequences, which can be very long, a deep **1D Convolutional Neural Network (CNN)** with **Residual Blocks** is implemented. This architecture is highly effective at learning hierarchical features and local patterns (motifs) in the amino acid sequence.

* **Residual Connections:** These "shortcut" connections are crucial for training deep networks, as they prevent the vanishing gradient problem and allow for more effective learning.
* **Batch Normalization:** Used after each convolution to stabilize and accelerate the training process.

The encoder processes the sequence and uses an adaptive pooling layer to produce a fixed-size embedding vector.

#### 2.3 Main Interaction Model

This final module brings the two encoders together.

1.  It takes the drug graph and protein sequence as input.
2.  It passes each input through its respective pre-trained encoder to obtain two 128-dimensional embedding vectors.
3.  These two vectors are **concatenated** to form a single 256-dimensional feature vector.
4.  This combined vector is then fed into a deep, multi-layer perceptron (the **classifier head**) which has been fine-tuned on a synthetically balanced dataset to predict the final binding affinity.

In [ ]:
class DrugEncoderGAT(nn.Module):
    """
    A Graph Attention Network (GAT) to encode drug molecules.
    """
    def __init__(self, num_node_features, embedding_dim=128, heads=8):
        """
        Initializes the GAT layers.

        Args:
            num_node_features (int): The number of features for each atom (e.g., 5).
            embedding_dim (int): The size of the final output embedding vector.
            heads (int): The number of attention heads to use in GAT layers.
        """
        super(DrugEncoderGAT, self).__init__()

        # GAT layers
        # The first layer takes the raw node features
        self.gat1 = GATConv(num_node_features, 32, heads=heads)

        # The second layer takes the concatenated output of the first layer's heads
        self.gat2 = GATConv(32 * heads, 64, heads=heads)

        # The final GAT layer. We use 1 head to get a single output vector.
        self.gat3 = GATConv(64 * heads, embedding_dim, heads=1)

    def forward(self, data):
        """
        Defines the forward pass of the model.
        """
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Pass through GAT layers with ELU activation (common for GAT)
        x = F.elu(self.gat1(x, edge_index))
        x = F.elu(self.gat2(x, edge_index))
        x = self.gat3(x, edge_index) # No activation on the final GAT layer before pooling

        # Apply global mean pooling
        graph_embedding = global_mean_pool(x, batch)

        return graph_embedding


print("--- GAT-based Drug Encoder ---")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
class ResidualBlock(nn.Module):
    """A simple residual block for 1D CNN."""
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding='same')
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding='same')
        self.bn2 = nn.BatchNorm1d(out_channels)

        # A projection layer in case input and output dimensions don't match
        self.shortcut = nn.Identity()
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = self.shortcut(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual # The residual connection
        return F.relu(out)

class ProteinEncoderDeeper(nn.Module):
    """
    A deeper and more powerful 1D-CNN for protein sequences, using residual blocks.
    """
    def __init__(self, vocab_size, embedding_dim=128, amino_acid_embed_size=128):
        super(ProteinEncoderDeeper, self).__init__()

        self.embedding = nn.Embedding(vocab_size, amino_acid_embed_size, padding_idx=0)

        # A sequence of residual blocks to increase depth and power
        self.res_block1 = ResidualBlock(amino_acid_embed_size, 128)
        self.res_block2 = ResidualBlock(128, 256)
        self.res_block3 = ResidualBlock(256, embedding_dim)

        self.pool = nn.AdaptiveMaxPool1d(1)

    def forward(self, sequence_tokens):
        x = self.embedding(sequence_tokens)
        x = x.permute(0, 2, 1) # Reshape for Conv1d

        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)

        x = self.pool(x)
        final_embedding = x.squeeze(2)

        return final_embedding

In [ ]:
VOCAB = "XACDEFGHIKLMNPQRSTVWY"
char_to_int = {char: i for i, char in enumerate(VOCAB)}
vocab_size = len(VOCAB)


class DTI_Model_Deeper(nn.Module):
    def __init__(self, drug_encoder, protein_encoder, min_val=5.0, max_val=11.0):
        super(DTI_Model_Deeper, self).__init__()

        self.drug_encoder = drug_encoder
        self.protein_encoder = protein_encoder

        # Classifier head remains the same
        self.classifier = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        # Store min and max values for scaling
        self.min_val = min_val
        self.max_val = max_val

    def forward(self, drug_data, protein_data):
        drug_embedding = self.drug_encoder(drug_data)
        protein_embedding = self.protein_encoder(protein_data)

        combined_features = torch.cat([drug_embedding, protein_embedding], dim=1)

        # Get the raw output from the classifier
        raw_output = self.classifier(combined_features)
        scaled_output = torch.sigmoid(raw_output) * (self.max_val - self.min_val) + self.min_val

        return scaled_output


### Part 3: Advanced Training - Two-Stage Fine-Tuning with Synthetic Data

After observing that even a powerful model architecture struggled with the imbalanced nature of the Davis dataset, we implemented a sophisticated, data-centric training strategy. This workflow is divided into four main stages to specifically address the lack of high-affinity samples.

#### Stage 1: Pre-training the Encoders
The full DTI model, including the GAT drug encoder and the deep residual CNN protein encoder, is trained for a limited number of epochs (e.g., 10-15) on the original, imbalanced regression dataset. 

**Goal:** The purpose of this stage is not to create a perfect final model, but to train the encoders to a point where they can generate meaningful and high-quality numerical representations (embeddings) from the raw molecular and protein data. The weights from this stage are saved.

#### Stage 2: Extracting the Embedding Space
The pre-trained encoders are then used to process the entire training dataset. We pass all drugs and proteins through their respective encoders and save the resulting 128-dimensional embedding vectors to a new file.

**Goal:** This converts our complex, multi-modal dataset of graphs and sequences into a simple, purely numerical dataset (the "embedding space"), which is much easier to manipulate and analyze.

#### Stage 3: Synthetic Data Generation (SMOTE for Regression)
This is the core of our data balancing strategy. We load the extracted embeddings and apply a fast, manual **SMOTE-like algorithm** for regression.

**Goal:** To fix the data imbalance by generating new, synthetic data points that are representative of the rare, high-affinity samples. The script identifies the "minority class" (samples with $pK_d > 7.5$) and creates new samples by interpolating between multiple random parents from this class. The result is a new, much larger, and more balanced dataset, which is then saved.

#### Stage 4: Fine-tuning the Classifier Head
This is the final training phase.
1.  **Load Models:** The full DTI model architecture is instantiated again, and the saved pre-trained weights from Stage 1 are loaded.
2.  **Freeze Encoders:** The weights of the drug and protein encoders are **frozen** (`requires_grad = False`).
3.  **Train Classifier:** Only the final classifier head (the MLP that makes the decision) is trained on our new, high-quality, balanced dataset of embeddings.

**Goal:** To fine-tune the decision-making part of our network using the improved dataset, without altering the powerful feature extractors we've already trained. This is a highly effective technique for leveraging pre-training and improving performance on skewed datasets.

In [ ]:
print("--- Starting Step 1: Pre-training Encoders ---")

# Load regression dataset
full_dataset = torch.load('davis_regression_dataset.pt', weights_only=False)
train_dataset, test_dataset = train_test_split(full_dataset, test_size=0.2, random_state=42)
train_loader = PyGDataLoader(train_dataset, batch_size=128, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

drug_encoder = DrugEncoderGAT(num_node_features=5, embedding_dim=128).to(device)
protein_encoder = ProteinEncoderDeeper(vocab_size=len(VOCAB), embedding_dim=128).to(device)
model = DTI_Model_Deeper(drug_encoder, protein_encoder).to(device)

loss_fn = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)

# Train for limited epochs
num_pretrain_epochs = 10
for epoch in range(num_pretrain_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Pre-training Epoch {epoch+1}/{num_pretrain_epochs}"):
        batch = batch.to(device)
        sequences = batch.sequence
        tokenized = [torch.tensor([char_to_int.get(c, 0) for c in s], dtype=torch.long) for s in sequences]
        padded = torch.nn.utils.rnn.pad_sequence(tokenized, batch_first=True, padding_value=0).to(device)
        labels = batch.y.to(device)

        outputs = model(batch, padded)
        loss = loss_fn(outputs.squeeze(), labels.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print("\nEncoder pre-training complete.")
torch.save(model.state_dict(), 'pretrained_dti_model.pth')
print("Pre-trained model saved to 'pretrained_dti_model.pth'")


print("\n--- Starting Step 2: Extracting Embeddings ---")

model.eval()

all_drug_embeddings = []
all_protein_embeddings = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(train_loader, desc="Extracting Embeddings"):
        batch = batch.to(device)
        sequences = batch.sequence
        tokenized = [torch.tensor([char_to_int.get(c, 0) for c in s], dtype=torch.long) for s in sequences]
        padded = torch.nn.utils.rnn.pad_sequence(tokenized, batch_first=True, padding_value=0).to(device)

        drug_emb = model.drug_encoder(batch)
        protein_emb = model.protein_encoder(padded)

        all_drug_embeddings.append(drug_emb.cpu())
        all_protein_embeddings.append(protein_emb.cpu())
        all_labels.append(batch.y.cpu())

drug_embeddings_tensor = torch.cat(all_drug_embeddings, dim=0)
protein_embeddings_tensor = torch.cat(all_protein_embeddings, dim=0)
labels_tensor = torch.cat(all_labels, dim=0)

torch.save({
    'drug_embeddings': drug_embeddings_tensor,
    'protein_embeddings': protein_embeddings_tensor,
    'labels': labels_tensor
}, 'embedded_dataset.pt')

print("\nEmbedding extraction complete.")
print(f"Final dataset shapes: Drug Embeddings: {drug_embeddings_tensor.shape}, Protein Embeddings: {protein_embeddings_tensor.shape}, Labels: {labels_tensor.shape}")
print("Embedded dataset saved to 'embedded_dataset.pt'")

In [ ]:
print("--- Starting Final Step (Advanced SMOTE for Regression) ---")
try:
    # 1. بارگذاری دیتاست Embedding
    # 1. Load embedding dataset
    embedded_data = torch.load('embedded_dataset.pt')
    features = np.concatenate([
        embedded_data['drug_embeddings'].numpy(),
        embedded_data['protein_embeddings'].numpy()
    ], axis=1)
    labels = embedded_data['labels'].numpy().flatten()

    # 2. Seperate minority samples
    threshold = 6
    minority_indices = np.where(labels >= threshold)[0]
    minority_features = features[minority_indices]
    minority_labels = labels[minority_indices]

    print(f"Found {len(minority_labels)} minority samples (affinity >= {threshold}).")

    # --- KEY CHANGE 1: Control the number of synthetic samples ---
    # We will create N times the number of original minority samples.
    # You can change this factor. 2.5 means we are heavily oversampling.
    oversampling_factor = 2.5
    num_synthetic_samples = int(len(minority_labels) * oversampling_factor)

    print(f"Generating {num_synthetic_samples} new synthetic samples using 4 parents...")
    synthetic_features = []
    synthetic_labels = []

    for _ in tqdm(range(num_synthetic_samples)):
        # --- KEY CHANGE 2: Select 4 random parents from the minority class ---
        parent_indices = np.random.choice(len(minority_labels), 6, replace=True)

        # Calculate the average of the features and labels of the 4 parents
        parent_features = minority_features[parent_indices]
        parent_labels = minority_labels[parent_indices]

        new_features = parent_features.mean(axis=0)
        new_label = parent_labels.mean()

        synthetic_features.append(new_features)
        synthetic_labels.append(new_label)

    # 4. Merge synthetic dataset into initial dataset
    final_features = np.concatenate([features, np.array(synthetic_features)])
    final_labels = np.concatenate([labels, np.array(synthetic_labels)])

    # Convert to torch tensors
    X_balanced_tensor = torch.tensor(final_features, dtype=torch.float)
    y_balanced_tensor = torch.tensor(final_labels, dtype=torch.float).unsqueeze(1)

    # Save final dataset 
    torch.save({
        'features': X_balanced_tensor,
        'labels': y_balanced_tensor
    }, 'final_balanced_dataset.pt')

    print("\nFinal balanced dataset saved to 'final_balanced_dataset.pt'")
    print("New data shape:", X_balanced_tensor.shape)

    # Visualize Distribution
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.hist(labels, bins=30, color='blue', alpha=0.7)
    plt.title('Original Data Distribution')
    plt.xlabel('Affinity')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.hist(final_labels, bins=30, color='green', alpha=0.7)
    plt.title('Balanced Data Distribution (after Manual SMOTE)')
    plt.xlabel('Affinity')

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
print("--- Starting Final Step 4: Fine-tuning the Classifier ---")
try:
    # 1. Load final dataset
    balanced_data = torch.load('final_balanced_dataset.pt')
    features = balanced_data['features']
    labels = balanced_data['labels']

    # 2. Create standard dataset and dataloader of pytorch 
    full_dataset = TensorDataset(features, labels)
    train_dataset, test_dataset = train_test_split(full_dataset, test_size=0.2, random_state=42)

    train_loader = TorchDataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = TorchDataLoader(test_dataset, batch_size=64, shuffle=False)
    print("Standard torch DataLoaders created successfully.")

    # 3. preparing model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # create DTI model with final architecture
    drug_encoder = DrugEncoderGAT(num_node_features=5, embedding_dim=128)
    protein_encoder = ProteinEncoderDeeper(vocab_size=len(VOCAB), embedding_dim=128)
    model = DTI_Model_Deeper(drug_encoder, protein_encoder).to(device)

    # Load pre-trained model weights
    model.load_state_dict(torch.load('pretrained_dti_model.pth'))
    print("Pre-trained model weights loaded.")

    # Freezing encoder layers
    for param in model.drug_encoder.parameters():
        param.requires_grad = False
    for param in model.protein_encoder.parameters():
        param.requires_grad = False
    print("Encoder layers have been frozen.")

    # 4. preparing optimizer (only for classifier parameters)
    optimizer = Adam(model.classifier.parameters(), lr=0.0001, weight_decay=1e-5)
    loss_fn = nn.MSELoss()
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=5, verbose=True)

    # 5. final training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch_features, batch_labels in tqdm(train_loader, desc=f"Fine-tuning Epoch {epoch+1}/{num_epochs}"):
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            outputs = model.classifier(batch_features)

            loss = loss_fn(outputs.squeeze(), batch_labels.squeeze())

            total_train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = total_train_loss / len(train_loader)

        # evaluation
        model.eval()
        total_test_loss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch_features, batch_labels in test_loader:
                batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
                outputs = model.classifier(batch_features)
                test_loss = loss_fn(outputs.squeeze(), batch_labels.squeeze())
                total_test_loss += test_loss.item()
                all_preds.append(outputs.cpu())
                all_labels.append(batch_labels.cpu())

        avg_test_loss = total_test_loss / len(test_loader)
        rmse = np.sqrt(avg_test_loss)
        test_preds = torch.cat(all_preds).squeeze().numpy()
        test_labels = torch.cat(all_labels).squeeze().numpy()
        pearson_corr, _ = pearsonr(test_labels, test_preds)

        print(f"Epoch {epoch+1}/{num_epochs} | Train MSE: {avg_train_loss:.4f} | "
              f"Test MSE: {avg_test_loss:.4f} | Pearson Corr: {pearson_corr:.4f}")
        scheduler.step(avg_test_loss)

except Exception as e:
    print(f"An error occurred: {e}")

### Part 5: Final Performance Visualization

The final step is to evaluate our fine-tuned model on the held-out test set and visualize its performance. The code below generates a **scatter plot** to compare the model's predicted binding affinities against the true values.

This visualization provides an intuitive assessment of the model's success:
* **X-Axis:** True Affinity Values ($pK_d$)
* **Y-Axis:** Predicted Affinity Values ($pK_d$)
* **Red Dashed Line:** Represents a perfect prediction where `predicted = true`.

The closer the blue data points cluster around the red line, the more accurate the model. This plot visually confirms the high **Pearson Correlation (~0.82)** achieved after our advanced, data-centric training process, marking a successful conclusion to the project.

In [ ]:
# --- Plotting Predicted vs. True Values ---
plt.figure(figsize=(8, 8))

# Create the scatter plot
plt.scatter(test_labels, test_preds, alpha=0.5, label='Predictions')

# Add the perfect prediction line (y=x) for reference
min_val = min(test_labels.min(), test_preds.min())
max_val = max(test_labels.max(), test_preds.max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', linewidth=2, label='Perfect Prediction (y=x)')

# Add labels and title
plt.xlabel('True Affinity Values ($pK_d$)', fontsize=12)
plt.ylabel('Predicted Affinity Values ($pK_d$)', fontsize=12)
plt.title(f'Model Performance on Test Set\nRMSE: {rmse:.4f} | Pearson: {pearson_corr:.4f}', fontsize=14)
plt.legend()
plt.grid(True)
plt.axis('equal') # Ensure the scale is the same on both axes
plt.show()

### Conclusion & Future Work

#### Conclusion

This project successfully demonstrates the end-to-end development of a sophisticated, multi-modal deep learning model for predicting drug-target binding affinity. Through an iterative process of architectural improvements, advanced training strategies, and a crucial data-centric approach to handle imbalance, the final model achieved a strong **Pearson Correlation of ~0.82** and an **RMSE of ~0.62** on the test set.

The key takeaway was the success of the two-stage fine-tuning workflow. By pre-training powerful encoders (GAT and ResNet-CNN) and then fine-tuning a classifier head on a synthetically balanced dataset of embeddings, we were able to overcome the model's initial conservative bias and significantly improve its predictive power on rare, high-affinity samples.

#### Future Work

While the final model is highly performant, its primary limitation remains the severe imbalance of the original Davis dataset. Future iterations of this project could explore the following avenues for further improvement:

* **Advanced Loss Functions:** Implement custom loss functions, such as Focal Loss adapted for regression, to be even more robust to the data skew during training.
* **Generalization to Other Datasets:** Evaluate the model's performance on other benchmark DTI datasets, such as KIBA, to test its generalization capabilities beyond the Davis dataset.
* **State-of-the-Art Protein Encoders:** Replace the 1D Residual CNN with a pre-trained, Transformer-based model (e.g., ESM-2) to potentially capture more complex, long-range dependencies within the protein sequences.

Overall, this project serves as a comprehensive case study in building and refining a deep learning solution for a challenging bioinformatics problem.